In [1]:
!pip install pandas 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

In [3]:
wine_df = pd.read_csv('winequality-red.csv', sep=";")
print(f"wine_df.shape: {wine_df.shape}")

wine_df.head()

wine_df.shape: (1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
wine_df.quality.unique()

array([5, 6, 7, 4, 8, 3])

In [5]:
# PERFORM A PREPROCESSING OF THE DATASET

In [6]:
y = wine_df.quality
X = wine_df.drop(columns = ['quality'])

In [7]:

X.shape, y.shape

((1599, 11), (1599,))

In [8]:
#from tf.keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical

WINE_QUALITIES = 11

yy = to_categorical(y, num_classes=WINE_QUALITIES)

yy.shape

2023-11-06 00:07:25.302099: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 00:07:25.351034: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 00:07:25.351065: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 00:07:25.351095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 00:07:25.360579: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-06 00:07:25.361422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

(1599, 11)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def make_model(input_shape, num_classes=WINE_QUALITIES):
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Dense(48, activation="relu"),
        layers.Dense(24, activation="relu"),
        layers.Dense(WINE_QUALITIES, activation="softmax"),
    ])

    return model

In [10]:
features = X.shape[1:]
model = make_model(input_shape=features)
#keras.utils.plot_model(model, show_shapes=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                576       
                                                                 
 dense_1 (Dense)             (None, 24)                1176      
                                                                 
 dense_2 (Dense)             (None, 11)                275       
                                                                 
Total params: 2027 (7.92 KB)
Trainable params: 2027 (7.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-11-06 00:07:27.577243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-11-06 00:07:27.577300: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: bell
2023-11-06 00:07:27.577315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: bell
2023-11-06 00:07:27.577510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 545.29.2
2023-11-06 00:07:27.577564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-06 00:07:27.577594: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 545.29.2 -- cannot find working devices in this configuration


In [11]:
epochs = 512

checkpoint_filepath = './checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_categorical_crossentropy',
    mode='max',
    save_best_only=True)

callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    model_checkpoint_callback,
    tf.keras.callbacks.ProgbarLogger(count_mode="steps"),
    #tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta=10**-3)
]

metrics = [
    #tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.CategoricalCrossentropy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.AUC(multi_label=True),
    #tf.keras.metrics.Recall()
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=metrics
)

In [12]:
model.fit(
    X,
    yy,
    epochs=epochs,
    callbacks=callbacks,
    validation_split = 0.2,
)

Epoch 1/512
40/40 [==============================] - 2s 14ms/step - loss: 2.9011 - categorical_crossentropy: 2.9011 - precision: 0.2673 - auc: 0.3032 - val_loss: 1.3119 - val_categorical_crossentropy: 1.3119 - val_precision: 0.5231 - val_auc: 0.3548
Epoch 2/512
40/40 [==============================] - 0s 3ms/step - loss: 1.3063 - categorical_crossentropy: 1.3063 - precision: 0.5652 - auc: 0.3323 - val_loss: 1.2214 - val_categorical_crossentropy: 1.2214 - val_precision: 0.5182 - val_auc: 0.3515
Epoch 3/512
40/40 [==============================] - 0s 3ms/step - loss: 1.2537 - categorical_crossentropy: 1.2537 - precision: 0.6139 - auc: 0.3392 - val_loss: 1.1825 - val_categorical_crossentropy: 1.1825 - val_precision: 0.5686 - val_auc: 0.3536
Epoch 4/512
40/40 [==============================] - 0s 3ms/step - loss: 1.2242 - categorical_crossentropy: 1.2242 - precision: 0.6014 - auc: 0.3363 - val_loss: 1.1761 - val_categorical_crossentropy: 1.1761 - val_precision: 0.5238 - val_auc: 0.3586
Epo

In [20]:
sample = wine_df.sample(n=5)

sample

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
50,8.8,0.66,0.26,1.7,0.074,4.0,23.0,0.99710,3.15,0.74,9.2,5
655,9.7,0.55,0.17,2.9,0.087,20.0,53.0,1.00040,3.14,0.61,9.4,5
513,10.5,0.51,0.64,2.4,0.107,6.0,15.0,0.99730,3.09,0.66,11.8,7
1238,8.1,0.73,0.00,2.5,0.081,12.0,24.0,0.99798,3.38,0.46,9.6,4
897,7.1,0.59,0.01,2.3,0.080,27.0,43.0,0.99550,3.42,0.58,10.7,6


In [21]:
x_sample, y_sample = sample.drop(columns=['quality']), sample.quality

In [22]:
y_pred = model.predict(x_sample)

y_pred

1/1 [==============================] - 0s 27ms/step


array([[2.4058949e-08, 3.6080780e-10, 3.8770431e-10, 4.2056327e-04,
        4.8060913e-02, 5.4626954e-01, 3.8589856e-01, 1.8301027e-02,
        1.0494398e-03, 3.3733072e-10, 4.3668658e-10],
       [8.7271225e-11, 2.8584828e-12, 1.7474520e-12, 8.5580155e-05,
        2.6946271e-02, 5.3493088e-01, 4.2750478e-01, 1.0519725e-02,
        1.2775780e-05, 3.1112747e-12, 7.7276510e-13],
       [1.6543258e-10, 1.1301071e-11, 2.1287234e-13, 2.6824669e-06,
        2.5933504e-03, 5.7702545e-02, 8.1830060e-01, 8.7724447e-02,
        3.3676382e-02, 2.8183152e-13, 1.6652620e-12],
       [9.0382812e-09, 2.8595548e-10, 3.3591957e-10, 3.8468572e-03,
        1.7572300e-01, 6.3926214e-01, 1.5537827e-01, 2.5765808e-02,
        2.4001243e-05, 6.3329925e-10, 1.3644248e-10],
       [4.0541291e-08, 1.2407888e-08, 2.6043245e-09, 2.6167030e-05,
        2.6916552e-02, 3.5008195e-01, 5.2923423e-01, 9.2521735e-02,
        1.2193316e-03, 2.9738063e-09, 2.4033473e-09]], dtype=float32)

In [23]:
import numpy as np

np.argmax(y_pred, axis=1)

array([5, 5, 6, 5, 6])